## 데이터 준비와 전처리

In [203]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [204]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [205]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [206]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head(15)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


## 분석해 봅시다.

In [207]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [208]:
#rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [209]:
# movies와 ratings를 합쳐준다.
data= pd.merge(movies, ratings)

In [210]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = [ 'user_id','title', 'rating']
data = data[using_cols]
data.head(10)

,user_id,title,rating
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
5,18,Toy Story (1995),4
6,19,Toy Story (1995),5
7,21,Toy Story (1995),3
8,23,Toy Story (1995),4
9,26,Toy Story (1995),3


In [211]:
data['title'] = data['title'].str.lower() 

In [212]:
# 잘 합쳐졌는 지 확인
movies_ratings.head(5)

,movie_id,title,genre,user_id,count,timestamp,rating
0,1.0,Toy Story (1995),Animation|Children's|Comedy,1,5.0,978824268.0,NaN
1,1.0,Toy Story (1995),Animation|Children's|Comedy,6,4.0,978237008.0,NaN
2,1.0,Toy Story (1995),Animation|Children's|Comedy,8,4.0,978233496.0,NaN
3,1.0,Toy Story (1995),Animation|Children's|Comedy,9,5.0,978225952.0,NaN
4,1.0,Toy Story (1995),Animation|Children's|Comedy,10,5.0,978226474.0,NaN


In [213]:
#가장 인기있는 영화 30개(인기순)
movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [214]:
# 유저별 몇 편의 영화를 보고 있는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

## 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [215]:
# 내가 좋아하시는 영화의 데이터로 바꿔서 추가한다.
my_favorite = ['men in black (1997)' , 'fugitive, the (1993)' ,'terminator, the (1984)' ,
               'back to the future (1985)' ,'forrest gump (1994)']

# 'kimin'이라는 user_id가 위 영화의 평점을 5점씩 주었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['kimin']*5, 'title': my_favorite, 'rating':[5]*5})

# user_id에 'kimin'이라는 데이터가 없다면
# 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
if not data.isin({'user_id':['kimin']})['user_id'].any():  
    data = data.append(my_playlist)   
    
data.tail(20) 

,user_id,title,rating
836463,5074,"contender, the (2000)",4
836464,5087,"contender, the (2000)",4
836465,5100,"contender, the (2000)",4
836466,5205,"contender, the (2000)",4
836467,5304,"contender, the (2000)",4
836468,5333,"contender, the (2000)",4
836469,5359,"contender, the (2000)",5
836470,5405,"contender, the (2000)",4
836471,5475,"contender, the (2000)",5
836472,5602,"contender, the (2000)",3


## 모델에 활용하기 위한 전처리

In [216]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 타이틀 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [217]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['kimin'])    # 60340명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(title_to_idx['forrest gump (1994)'])

6039
342


In [218]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,rating
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,1419,5
1,6039,439,5
2,6039,1122,5
3,6039,1152,5


## CSR matrix를 직접 만들어 봅시다.

In [219]:
data['rating']

0    5
1    4
2    4
3    5
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: rating, Length: 836483, dtype: int64

In [220]:
print(data.rating)

0    5
1    4
2    4
3    5
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: rating, Length: 836483, dtype: int64


In [221]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.rating, (data.user_id, data.title)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [222]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [276]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [277]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [278]:
als_model.fit(csr_data_transpose)

In [279]:
kimin, back_to_the_future = user_to_idx['kimin'], title_to_idx['back to the future (1985)']
kimin_vector, back_to_the_future_vector = als_model.user_factors[kimin], als_model.item_factors[back_to_the_future]

In [280]:
kimin_vector

array([-0.13527389, -0.27319938, -0.08741796,  0.11758536,  0.36316   ,
       -0.3983217 ,  0.8465977 ,  0.26570088,  0.28809384,  0.41657785,
       -0.779565  , -0.15212072,  0.51097274, -0.08541659,  0.02522049,
       -0.23541461,  0.74912596, -1.3009206 , -0.28464815, -0.09308033,
        0.13687326, -0.05327605, -0.09837456,  0.02233899,  0.6462414 ,
        0.42036277,  0.02123069,  0.16627568, -0.94141316,  0.05665561,
        0.979777  , -0.12404861,  0.04477061,  0.52959156, -0.1846351 ,
        0.5908083 ,  0.4548125 , -0.10116755,  0.2925298 ,  0.9393734 ,
       -0.16464   , -0.14681849, -0.7219284 , -0.03014675, -0.27009854,
       -0.10670654,  0.10798036, -0.47603264,  0.34591466,  0.09919448,
       -0.10908444, -0.27620566,  0.8802672 , -0.56187224, -0.5249028 ,
        0.13426025,  0.32797465,  0.16148935, -0.0992355 , -0.24543321,
       -0.02871619, -0.0884739 , -0.03289748, -0.15102385, -0.52618265,
       -0.25174585, -0.13285111,  0.37412968,  0.10310774,  0.43

In [281]:
back_to_the_future_vector

array([ 3.92493010e-02,  1.06060123e-02,  7.16290344e-03,  2.26534531e-03,
        3.25448513e-02, -8.49427097e-03,  3.58712114e-02,  5.00428490e-03,
        1.54201267e-02,  7.16679031e-03, -2.46587042e-02,  8.87601078e-03,
        2.40650233e-02, -1.75100230e-02,  4.56794025e-03,  1.52589623e-02,
        4.72448803e-02,  8.62265821e-04, -8.64874292e-03,  3.22266109e-03,
       -3.11191287e-02,  4.41815937e-03, -1.00368727e-02, -2.14320123e-02,
        3.17748636e-05, -3.43624233e-05,  3.24694254e-02,  1.74049754e-03,
       -1.13676852e-02, -3.64482589e-02,  3.39770764e-02, -1.59854628e-02,
       -3.98274307e-04,  3.96300554e-02, -1.31690921e-03,  5.70498034e-02,
        1.09002255e-02,  1.08091002e-02,  2.52412260e-02,  7.49133900e-03,
       -1.55015811e-02, -3.63593292e-03, -1.66432895e-02,  5.13732322e-02,
       -1.76583529e-02,  1.59106869e-02,  3.56787108e-02, -1.81110613e-02,
        8.65196530e-03,  3.84863317e-02, -9.15466156e-03, -2.45760009e-03,
        2.45960783e-02, -

## 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [282]:
np.dot(kimin_vector, back_to_the_future_vector)

0.61317134

In [283]:
toy = title_to_idx['toy story (1995)']
toy_vector = als_model.item_factors[toy]
np.dot(kimin_vector, toy_vector)

0.23128474

## 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [248]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [241]:
get_similar_title('men in black (1997)') 

['men in black (1997)',
 'jurassic park (1993)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'independence day (id4) (1996)',
 'matrix, the (1999)',
 'fifth element, the (1997)',
 'face/off (1997)',
 'lost world: jurassic park, the (1997)',
 'schlafes bruder (brother of sleep) (1995)']

In [242]:
get_similar_title('toy story (1995)') 

['toy story (1995)',
 'toy story 2 (1999)',
 "bug's life, a (1998)",
 'aladdin (1992)',
 'babe (1995)',
 'groundhog day (1993)',
 'pleasantville (1998)',
 'lion king, the (1994)',
 'forrest gump (1994)',
 'beauty and the beast (1991)']

In [243]:
get_similar_title('back to the future (1985)') 

['back to the future (1985)',
 "ferris bueller's day off (1986)",
 'back to the future part ii (1989)',
 'when harry met sally... (1989)',
 'ghostbusters (1984)',
 'e.t. the extra-terrestrial (1982)',
 'big (1988)',
 'fish called wanda, a (1988)',
 'cocoon (1985)',
 'airplane! (1980)']

## 내가 가장 좋아할 만한 영화들을 추천받기.

In [275]:
user = user_to_idx['kimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
for i in title_recommended: 
    print("{} {:08.3f}".format(idx_to_title[i[0]], i[1]))

terminator 2: judgment day (1991) 0000.614
jurassic park (1993) 0000.523
matrix, the (1999) 0000.516
star wars: episode vi - return of the jedi (1983) 0000.391
total recall (1990) 0000.380
braveheart (1995) 0000.353
hunt for red october, the (1990) 0000.350
star wars: episode iv - a new hope (1977) 0000.340
star wars: episode v - the empire strikes back (1980) 0000.334
groundhog day (1993) 0000.333
die hard (1988) 0000.327
saving private ryan (1998) 0000.301
speed (1994) 0000.287
raiders of the lost ark (1981) 0000.261
toy story (1995) 0000.259
sixth sense, the (1999) 0000.255
e.t. the extra-terrestrial (1982) 0000.250
airplane! (1980) 0000.237
independence day (id4) (1996) 0000.231
indiana jones and the last crusade (1989) 0000.215


In [251]:
rihanna = title_to_idx['die hard (1988)']
explain = als_model.explain(user, csr_data, itemid=rihanna)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('terminator, the (1984)', 0.199224832553194),
 ('fugitive, the (1993)', 0.15878254999424274),
 ('back to the future (1985)', 0.02165633232228849),
 ('forrest gump (1994)', 0.006815673903585511),
 ('men in black (1997)', -0.06310753641801863)]

## 마무리

이번 프로젝트는 추천 시스템에 대한 내용이었다.  
사용자가 선호하는 항목을 토대로 유사한 항목을 추천하는 시스템이다.  
추천 시스템의 기술로는 협업 필터링, 콘텐츠 기반 필터링이 있다.  
여기서는 비슷한 영화를 추천하거나 도출하는 활동만 있는 것으로 봐서 콘텐츠 기반 필터링만 있는 듯 하다.  
여기서는 특이하게 csv파일을 두개 받았다.  
내게 필요한 칼럼은 양쪽 모두에 있어서 하나만 쓸 수 없었다.  
어떻게 사용할까 고민하다가 전에 배운 merge를 사용했다.  
모델을 성공적으로 돌리니까 추천 시스템이 정상적으로 동작했다.  
iterations를 높일 수록 내가 선택한 영화와 행렬곱을 한 결과가 높아졌다.  
높은 정확도가 나오면 좋을 것 같지만, 내가 선호하는 다른 영화의 정확도는 떨어지는 단점이 있다.  
이 두가지를 적절하게 조절하는 시스템이 좋은 추천 시스템인듯 하다. 